In [1]:
!pip install -q -U langchain langchain-core langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==

In [2]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 7.1 MB/s eta 0:00:00


In [3]:
from langchain_core.prompts import PromptTemplate

In [4]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [5]:
from langchain_community.document_loaders import PyPDFLoader

In [6]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

In [8]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [9]:
def Get_pdf_text(path:dict)->str:
  if path!=None:
    pdf_url=path['input']
  else:
    pdf_url='/content/ukpga_20250022_en.pdf'

  pdf_loader=PyPDFLoader(pdf_url)
  pages=pdf_loader.load()
  full_text=''
  for p in pages:
    full_text+=p.page_content
  return full_text

In [10]:
Get_text=RunnableLambda(Get_pdf_text)

In [11]:
from google.colab import userdata
api_key=userdata.get('PROJECT')
llm=HuggingFaceEndpoint(
    repo_id='openai/gpt-oss-20b',
    huggingfacehub_api_token=api_key,
    temperature=0.2
        )

In [12]:
llm_chat=ChatHuggingFace(llm=llm)

In [13]:
summary_template='''
Your a halpful assistance.
Task:Summarize the given Act context.

Instructions:
1.Focus on the following points in the summary:
- Purpose
- Key definitions
- Eligibility
- Obligations
- Enforcement elements
2. Pay attention that the summary should not be less than 5 bullet points and not exceed more than 10 bullet points.

Fromat:Json.
Example:{{
  'summary_points':'['1.Point1', '2.Point2'.......'Point10']'}}

Context:
{context}
'''

In [14]:
summary_prompt=PromptTemplate(template=summary_template)

In [15]:
summary_chain=(
    summary_prompt
    | llm_chat
    | JsonOutputParser()
)

In [16]:
KeyExtractions_template='''
Your a helpful assitant.
Task: Extract key Legislative Sections from the given Act context.

Instructions:
1. The following Key should be extracted:
- Definitions
- Obligations
- Responsibilities
- Eligibility
- Payments / Entitlements
- Penalties / Enforcement
- Record-keeping / Reporting

2. Pay attention the result should be strictly in the Json format.

Fromat: Json with the following keys:
-definations
-obligations
-responsibilities
-eligibility
-payments
-panalties
-record_keeping

Context:
{context}
'''

In [17]:
KeyExtractions_prompt=PromptTemplate(template=KeyExtractions_template)

In [18]:

KeyExtraction_chain=(
     KeyExtractions_prompt
    | llm_chat
    | JsonOutputParser()
)

In [19]:
RuleCheck_template='''
Your a smart and helpful assistant.
Task:Appy the following rule checks on the given Act context.
Instructions:
1. The following rules should be checked:
-Act must define key terms
-Act must specify eligibility criteria
-Act must specify responsibilities of the administering authority
-Act must include enforcement or penalties
-Act must include payment calculation or entitlement structure
-Act must include record-keeping or reporting requirements

2. Pay attention the result should be strictly in the Json format.

Fromat: For each rule check the Json should consist the following keys:
-rule
-status
-evidence
-confidence

Example for one such rule Check is:
{{
 "rule": "Act must define key terms",
 "status": "pass",
 "evidence": "Section 2 – Definitions",
 "confidence": 92

}}

Act Context:
{Context}
'''

In [20]:
RuleCheck_prompt=PromptTemplate(template=RuleCheck_template)

In [21]:
RuleCheck_chain=(
     RuleCheck_prompt
    | llm_chat
    | JsonOutputParser()
)

In [22]:
parallel_chains=RunnableParallel(
    summary=summary_chain,
    key_extraction=KeyExtraction_chain,
    rule_check=RuleCheck_chain
)

In [23]:
Full_chain=(
    Get_text
    | parallel_chains
)

In [25]:
Full_chain.invoke({'input':'/content/ukpga_20250022_en.pdf'})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'summary': {'summary_points': ['1. Purpose: Adjust the rates of the standard allowance, the LCWRA element, the LCW element, and ESA IR components for the tax years 2026‑29.',
   "2. Key definitions: includes 'standard allowance', 'LCWRA element', 'pre‑2026 claimant', 'severe conditions criteria claimant', and 'terminally ill'.",
   '3. Eligibility: Applies to claimants with limited capability for work and work‑related activity, including pre‑2026, severe conditions criteria, or terminally ill claimants.',
   '4. Obligations of the Secretary of State: Must exercise statutory powers to set minimum rates, freeze the LCWRA/LCW elements, and adjust protected LCWRA or standard allowance amounts in line with CPI and uplift percentages.',
   '5. Enforcement: Overrides the usual annual up‑rating provisions of the Social Security Administration Act 1992 for the specified tax years, ensuring rates are maintained by statutory powers.',
   '6. Legacy ESA IR provisions: Similar CPI‑based adjustment